#   Outdoor Wheather

In [1]:
import matplotlib
%matplotlib inline
import numpy as np
import pandas as pd


In [2]:

!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas


import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)


  Using cached openmeteo_requests-1.2.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached openmeteo_sdk-1.11.3-py3-none-any.whl.metadata (13 kB)
  Using cached flatbuffers-24.3.7-py2.py3-none-any.whl.metadata (849 bytes)
Using cached openmeteo_requests-1.2.0-py3-none-any.whl (5.5 kB)
Using cached openmeteo_sdk-1.11.3-py3-none-any.whl (12 kB)
Using cached flatbuffers-24.3.7-py2.py3-none-any.whl (26 kB)
  Using cached requests_cache-1.2.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached retry_requests-2.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached cattrs-23.2.3-py3-none-any.whl.metadata (10 kB)
  Using cached url_normalize-1.4.3-py2.py3-none-any.whl.metadata (3.1 kB)
Using cached requests_cache-1.2.0-py3-none-any.whl (61 kB)
Using cached retry_requests-2.0.0-py3-none-any.whl (15 kB)
Using cached cattrs-23.2.3-py3-none-any.whl (57 kB)
Using cached url_normalize-1.4.3-py2.py3-none-any.whl (6.8 kB)


In [3]:


def make_dataframe(openmeteo_response):

    hourly = openmeteo_response.Hourly()
    _start = pd.to_datetime(hourly.Time(), unit = "s", utc = True)
    _end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True)
    _delta = pd.Timedelta(seconds = hourly.Interval())

    data = {
        "date": pd.date_range(_start, _end, freq=_delta, inclusive="left"),
        "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
        "surface_pressure": hourly.Variables(1).ValuesAsNumpy(),
        "wind_speed_10m": hourly.Variables(2).ValuesAsNumpy(),
        "wind_direction_10m": hourly.Variables(3).ValuesAsNumpy(),
        "wind_gusts_10m": hourly.Variables(4).ValuesAsNumpy(),
    }

    return pd.DataFrame(data).set_index("date")

In [8]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": [47.36, 46, 47.26, 46.5],
	"longitude": [8.55, 8.95, 11.39, 11.35],
	"start_date": "2010-01-01",
	"end_date": "2024-03-15",
	"hourly": ["temperature_2m", "surface_pressure", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m"]
}
data = openmeteo.weather_api(url, params=params)
data = dict(zip(("Zurich", "Lugano", "Innsbruck", "Bolzano"), data))
data = pd.concat({key: make_dataframe(val) for key, val in data.items()})

city_codes = {"Zurich": 1, "Lugano": 2, "Innsbruck": 3, "Bolzano": 4}
data['city_code'] = data.index.get_level_values(0).map(city_codes)

In [9]:
data

temperature_2m  surface_pressure  \
        date                                                          
Zurich  2010-01-01 00:00:00+00:00          0.8940        942.275085   
        2010-01-01 01:00:00+00:00          0.4940        941.824402   
        2010-01-01 02:00:00+00:00         -0.0060        941.640625   
        2010-01-01 03:00:00+00:00         -0.3060        941.112305   
        2010-01-01 04:00:00+00:00         -0.5560        940.593262   
...                                           ...               ...   
Bolzano 2024-03-15 19:00:00+00:00         14.6365        982.346619   
        2024-03-15 20:00:00+00:00         13.6365        982.812622   
        2024-03-15 21:00:00+00:00         12.7865        983.198181   
        2024-03-15 22:00:00+00:00         11.8365        983.764832   
        2024-03-15 23:00:00+00:00         10.6365        983.817932   

                                   wind_speed_10m  wind_direction_10m  \
        date                                                            
Zurich  2010-01-01 00:00:00+00:00        1.138420          341.564941   
        2010-01-01 01:00:00+00:00        1.018234           44.999897   
        2010-01-01 02:00:00+00:00        1.297998          123.690094   
        2010-01-01 03:00:00+00:00        1.800000          143.130020   
        2010-01-01 04:00:00+00:00        1.138420           71.564964   
...                                           ...                 ...   
Bolzano 2024-03-15 19:00:00+00:00        2.902413          240.255203   
        2024-03-15 20:00:00+00:00        3.671294          191.309891   
        2024-03-15 21:00:00+00:00        2.189795          189.462250   
        2024-03-15 22:00:00+00:00        1.018234          135.000107   
        2024-03-15 23:00:00+00:00        1.484318           75.963730   

                                   wind_gusts_10m  city_code  
        date                                                  
Zurich  2010-01-01 00:00:00+00:00        9.000000          1  
        2010-01-01 01:00:00+00:00        9.000000          1  
        2010-01-01 02:00:00+00:00        9.360000          1  
        2010-01-01 03:00:00+00:00        9.360000          1  
        2010-01-01 04:00:00+00:00       10.080000          1  
...                                           ...        ...  
Bolzano 2024-03-15 19:00:00+00:00       16.199999          4  
        2024-03-15 20:00:00+00:00       15.119999          4  
        2024-03-15 21:00:00+00:00       15.119999          4  
        2024-03-15 22:00:00+00:00       10.080000          4  
        2024-03-15 23:00:00+00:00        7.559999          4  

[498048 rows x 6 columns]

In [6]:
data.shape

(498048, 5)

In [7]:
data.value_counts()

temperature_2m  surface_pressure  wind_speed_10m  wind_direction_10m  wind_gusts_10m
 2.446000       980.534729        3.617955        5.710507            9.720000          2
 0.841500       948.469238        3.259938        353.659912          37.439999         2
-1.763500       986.544250        3.096837        324.462250          14.040000         2
 6.596000       993.695251        4.680000        360.000000          12.599999         2
 0.942500       944.773560        8.707238        172.875076          30.599998         2
                                                                                       ..
 7.292500       938.390869        6.638072        167.471207          30.599998         1
                938.297729        4.349896        155.556061          31.319998         1
                937.552307        4.693826        237.528824          29.160000         1
                936.900024        5.959060        295.016846          29.519999         1
 39.786503     

In [8]:
data.describe()

,temperature_2m,surface_pressure,wind_speed_10m,wind_direction_10m,wind_gusts_10m
count,350592.000000,350592.000000,350592.000000,350592.000000,350592.000000
mean,11.629037,970.744507,5.023917,171.817535,20.031143
std,8.358671,16.127691,3.217626,113.312103,11.000485
min,-21.107498,907.231873,0.000000,0.924034,1.440000
25%,5.242500,957.241333,2.902413,60.255207,12.240000
50%,11.583000,973.887726,4.379589,173.290253,17.280001
75%,17.941500,983.950012,6.287130,265.236450,24.840000
max,39.786503,1006.810303,40.711845,360.000000,132.839996


In [13]:
data.isnull()

temperature_2m  surface_pressure  \
        date                                                          
Zurich  2010-01-01 00:00:00+00:00           False             False   
        2010-01-01 01:00:00+00:00           False             False   
        2010-01-01 02:00:00+00:00           False             False   
        2010-01-01 03:00:00+00:00           False             False   
        2010-01-01 04:00:00+00:00           False             False   
...                                           ...               ...   
Bolzano 2019-12-31 19:00:00+00:00           False             False   
        2019-12-31 20:00:00+00:00           False             False   
        2019-12-31 21:00:00+00:00           False             False   
        2019-12-31 22:00:00+00:00           False             False   
        2019-12-31 23:00:00+00:00           False             False   

                                   wind_speed_10m  wind_direction_10m  \
        date                                                            
Zurich  2010-01-01 00:00:00+00:00           False               False   
        2010-01-01 01:00:00+00:00           False               False   
        2010-01-01 02:00:00+00:00           False               False   
        2010-01-01 03:00:00+00:00           False               False   
        2010-01-01 04:00:00+00:00           False               False   
...                                           ...                 ...   
Bolzano 2019-12-31 19:00:00+00:00           False               False   
        2019-12-31 20:00:00+00:00           False               False   
        2019-12-31 21:00:00+00:00           False               False   
        2019-12-31 22:00:00+00:00           False               False   
        2019-12-31 23:00:00+00:00           False               False   

                                   wind_gusts_10m  
        date                                       
Zurich  2010-01-01 00:00:00+00:00           False  
        2010-01-01 01:00:00+00:00           False  
        2010-01-01 02:00:00+00:00           False  
        2010-01-01 03:00:00+00:00           False  
        2010-01-01 04:00:00+00:00           False  
...                                           ...  
Bolzano 2019-12-31 19:00:00+00:00           False  
        2019-12-31 20:00:00+00:00           False  
        2019-12-31 21:00:00+00:00           False  
        2019-12-31 22:00:00+00:00           False  
        2019-12-31 23:00:00+00:00           False  

[350592 rows x 5 columns]

In [14]:
data.isna()

temperature_2m  surface_pressure  \
        date                                                          
Zurich  2010-01-01 00:00:00+00:00           False             False   
        2010-01-01 01:00:00+00:00           False             False   
        2010-01-01 02:00:00+00:00           False             False   
        2010-01-01 03:00:00+00:00           False             False   
        2010-01-01 04:00:00+00:00           False             False   
...                                           ...               ...   
Bolzano 2019-12-31 19:00:00+00:00           False             False   
        2019-12-31 20:00:00+00:00           False             False   
        2019-12-31 21:00:00+00:00           False             False   
        2019-12-31 22:00:00+00:00           False             False   
        2019-12-31 23:00:00+00:00           False             False   

                                   wind_speed_10m  wind_direction_10m  \
        date                                                            
Zurich  2010-01-01 00:00:00+00:00           False               False   
        2010-01-01 01:00:00+00:00           False               False   
        2010-01-01 02:00:00+00:00           False               False   
        2010-01-01 03:00:00+00:00           False               False   
        2010-01-01 04:00:00+00:00           False               False   
...                                           ...                 ...   
Bolzano 2019-12-31 19:00:00+00:00           False               False   
        2019-12-31 20:00:00+00:00           False               False   
        2019-12-31 21:00:00+00:00           False               False   
        2019-12-31 22:00:00+00:00           False               False   
        2019-12-31 23:00:00+00:00           False               False   

                                   wind_gusts_10m  
        date                                       
Zurich  2010-01-01 00:00:00+00:00           False  
        2010-01-01 01:00:00+00:00           False  
        2010-01-01 02:00:00+00:00           False  
        2010-01-01 03:00:00+00:00           False  
        2010-01-01 04:00:00+00:00           False  
...                                           ...  
Bolzano 2019-12-31 19:00:00+00:00           False  
        2019-12-31 20:00:00+00:00           False  
        2019-12-31 21:00:00+00:00           False  
        2019-12-31 22:00:00+00:00           False  
        2019-12-31 23:00:00+00:00           False  

[350592 rows x 5 columns]